# Basic Example
## Introduction

This notebook runs a single WattAdvisor optimization. 

The target is to find a cost minimal technology set for the supply of the given energy demands.

Copyright (c) 2007, Eclipse Foundation, Inc. and its licensors. All rights reserved.
Use of this source code is governed by a BSD-style license that can be found in the LICENSE file.

## Import of necessary packages
All necessary modules from WattAdvisor and additional packages are imported.

In [37]:
from pathlib import Path

from wattadvisor.utils.weather_data import (
    get_weather_data_from_era5_netcdf,
)
from wattadvisor.opt_model import OptModel
from wattadvisor.data_models.enums import EnergyType, EnergyUnit, EnergyPriceUnit
from wattadvisor.components import (
    energy_demand,
    heat_pump,
    photovoltaic,
    energy_purchase,
    gas_boiler,
    electrical_energy_storage,
    thermal_energy_storage,
)

## Specification of the considered location
At first, it should be specified at which location the plants should be dimensioned for. Therefore, the location is specified using decimal longitude and latitude geocoordinates.

In [38]:
latitude = 50.693
longitude = 10.937

## Load weather data file

In [39]:
path_netcdf = Path().joinpath("weather.nc")
weather_data = get_weather_data_from_era5_netcdf(
    path_netcdf, longitude=longitude, latitude=latitude
)

## Add energy demands

The following energy demands that should be supplied are added to the model:
- electrical energy demand with an annual demand of 3.500 kWh of electrical energy 
- thermal energy demand with an annual demand of 10.000 kWh of thermal energy 

The annual demand values given are used in combination with standard load profiles to create synthetic, hourly load profiles of the demands.

In [40]:
demands = [
    energy_demand.EnergyDemand(
        demand_sum=3500,
        demand_unit=EnergyUnit.KWH,
        energy_type=EnergyType.ELECTRICAL,
        profile_type="h0",
        profile_year=2022,
    ),
    energy_demand.EnergyDemand(
        demand_sum=10000,
        demand_unit=EnergyUnit.KWH,
        energy_type=EnergyType.THERMAL,
        profile_type="EFH",
        profile_year=2022,
        temperature_air=weather_data.air_temperature,
    )
]

## Add energy production, transformation and storage component options
To supply the energy demands, several component options are added to the model, from which the solver choses the optimal dimensioning:
- gas boiler
- air source heat pump
- ground source heat pump
- roof integrated photovoltaic plant
- wind power plant
- electrical energy storage
- thermal energy storage

In [41]:
components = [
    gas_boiler.GasBoiler(
        lifespan=20, eff=0.75, capex=300, opex=2, installed_power=10
    ),
    heat_pump.HeatPumpAir(
        lifespan=20,
        capex=1000,
        opex=2,
        source_temperature_series=weather_data.air_temperature,
    ),
    heat_pump.HeatPumpGround(
        lifespan=20,
        capex=1300,
        opex=2,
        source_temperature_series=weather_data.soil_temperature,
    ),
    photovoltaic.PhotovoltaikRoof(
        lifespan=20,
        capex=750,
        opex=2.87,
        potential_power=20,
        latitude=latitude,
        longitude=longitude,
        ghi=weather_data.ghi,
        dhi=weather_data.dhi,
        air_temperature=weather_data.air_temperature,
    ),
    electrical_energy_storage.ElectricalEnergyStorage(
        lifespan=15,
        capex_capacity=500,
        capex_power=500,
        opex=0.1,
        eff=0.9,
        relative_losses=0.00007
    ),
    thermal_energy_storage.ThermalEnergyStorage(
        lifespan=20,
        capex_capacity=5,
        capex_power=100,
        opex=2,
        eff=0.75,
        relative_losses=0.00138
    )
]

## Add energy purchases

Furthermore, options to purchase electrical energy and natural gas from external sources or public grid are added.

In [42]:
purchases = [
    energy_purchase.EnergyPurchase(
        name="Electrical energy purchase",
        energy_type=EnergyType.ELECTRICAL,
        energy_price_scalar=0.35,
        energy_price_unit=EnergyPriceUnit.EUR_PER_KWH,
        co2_intensity=445,
    ),
    energy_purchase.EnergyPurchase(
        name="Natural gas purchase",
        energy_type=EnergyType.NATURAL_GAS,
        energy_price_scalar=0.10,
        energy_price_unit=EnergyPriceUnit.EUR_PER_KWH,
        co2_intensity=202,
    ),
]

## Start optimization

The solver is to find the best solution for the model. This can take up to several minutes. 

In [43]:
options = demands + components + purchases

optimization = OptModel(options)

# Start the optimization
results = optimization.run_calculation()

2025-05-14 14:41:26,719 - INFO - Create model instance
2025-05-14 14:41:28,770 - INFO - Built bilance constraint for energy type EnergyType.ELECTRICAL.
2025-05-14 14:41:28,848 - INFO - Built bilance constraint for energy type EnergyType.THERMAL.
2025-05-14 14:41:28,907 - INFO - Built bilance constraint for energy type EnergyType.NATURAL_GAS.
2025-05-14 14:41:28,907 - INFO - Optimizing model
2025-05-14 14:41:28,977 - INFO - Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
2025-05-14 14:41:38,223 - INFO - RUN!
2025-05-14 14:41:38,225 - INFO - LP   has 227782 rows; 183992 cols; 582161 nonzeros
2025-05-14 14:41:38,230 - INFO - Coefficient ranges:
2025-05-14 14:41:38,230 - INFO -   Matrix [7e-05, 1e+03]
2025-05-14 14:41:38,230 - INFO -   Cost   [1e+00, 1e+00]
2025-05-14 14:41:38,230 - INFO -   Bound  [1e+01, 2e+01]
2025-05-14 14:41:38,230 - INFO -   RHS    [1e-02, 4e+00]
2025-05-14 14:41:38,325 - INFO - Presolving model
2025-05-14 14:41:38,457 - INF

## Inspect results

After optimization has finished, an object containing the relevant results is returned. 
For example, the advised power of each component, the total investment cost and the cost from energy purchase from external sources can be obtained from the object.

In [44]:
for component in results.target_scenario.components:
    if "advised_power" in component:
        print(f"{component['name']}: {component['advised_power']:.2f} kW")

    if "purchase_cost" in component:
        print(f"{component['name']}: {component['purchase_cost']:.2f} €/a")

print(f"Total investment cost: {results.target_scenario.kpis.total_investment_cost:.2f} €")

GasBoiler_2: 10.00 kW
HeatPumpAir_2: 0.42 kW
HeatPumpGround_2: 2.34 kW
PhotovoltaikRoof_2: 2.84 kW
ElectricalEnergyStorage_2: 1.03 kW
ThermalEnergyStorage_2: 0.89 kW
Electrical energy purchase: 969.70 €/a
Natural gas purchase: 18.09 €/a
Total investment cost: 11921.48 €


## Export results

Results are exported as a JSON file.

In [45]:
with open("results.json", "w") as file:
    file.write(results.model_dump_json(exclude_none=True, indent=4))